In [3]:
pip install Sastrawi

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Import library yang dibutuhkan
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
import pickle
from flask import Flask, request

# Membuat fungsi untuk text cleaning
def text_cleaning(text):
    # Lowercase teks
    text = text.lower()

    # Menghapus tanda baca dan angka
    text = re.sub(r'[^a-z]+', ' ', text)

    # Menghapus spasi yang berlebihan
    text = re.sub(' +', ' ', text)

    return text

# Membuat fungsi untuk stemming
def text_stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    text = stemmer.stem(text)

    return text

# Membaca data
df = pd.read_csv('data.csv')

# Melakukan text cleaning dan stemming pada data
df['Text'] = df['Text'].apply(text_cleaning)
df['Text'] = df['Text'].apply(text_stemming)

# Membagi data menjadi data latih dan data uji
X = df['Text'].values
y = df['Sentiment'].values

# Membuat object TfidfVectorizer
vectorizer = TfidfVectorizer()

# Transformasi teks menjadi representasi numerik (fitur)
X = vectorizer.fit_transform(X)

# Membagi data latih dan uji menjadi data latih dan data validasi
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Membuat object LogisticRegression
model = LogisticRegression(random_state=0)

# Melatih model
model.fit(X_train, y_train)

# Evaluasi model
y_pred = model.predict(X_test)

# Mencetak matriks konfusi dan laporan klasifikasi
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Menyimpan model menggunakan pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

# open model.pkl
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

[[1 0]
 [1 0]]
              precision    recall  f1-score   support

    negative       0.50      1.00      0.67         1
    positive       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/home/sam/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from flask import Flask, request
import pickle

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z]+', ' ', text)
    text = re.sub(' +', ' ', text)

    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = stemmer.stem(text)

    return text

app = Flask(__name__)

# Load the model from a file
with open("model.pkl", "rb") as file:
    model = pickle.load(file)

@app.route("/sentiment_analysis", methods=["POST"])
def sentiment_analysis():
    text = request.json["Text"]

    print(text)

    # Perform NLP operations
    processed_text = preprocess_text(text)
    sentiment = model.predict(processed_text)

    return {"sentiment": sentiment}

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2023-02-14 11:44:47,306] ERROR in app: Exception on /sentiment_analysis [POST]
Traceback (most recent call last):
  File "/home/sam/lib/python3.10/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/sam/lib/python3.10/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/sam/lib/python3.10/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/sam/lib/python3.10/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/tmp/ipykernel_29461/2537524619.py", line 28, in sentiment_analysis
    processed_text = preprocess_text(text)
  File "/tmp/ipykernel_29461/2537524619.py", line 6, in preprocess_text
    text = re.sub(r'[^a-z]+', ' ', text)
NameError: name 're' is not d

This is a great movie
